Spatial variable genes can be find in two different ways:

- Biased: Define spatial region by annotating (a.k.a glomeruli, cortex) and you compare region vs all of them.
- unbiased: looking for a pattern in the tissue, moransi algorithm

In [ ]:
#input dataset information by developers
plot_dir <- paste0(".//projects/kidney_mouse_ntn/reports/")
project_dir <- ".//Data/Spatial/visium/kidney/mouse_NTN/"
processed_dir <- ".//Data/Spatial/visium/kidney/mouse_NTN/processed/ultimate_processed/"
pipeline_dir <- ".//software/project_repos/visium_pipeline"
manifest_path <- paste0(project_dir, "/manifest.csv")
group_variable <- "diagnosis"

In [2]:
suppressWarnings(suppressMessages(library(Seurat)))
suppressWarnings(suppressMessages(library(ggplot2)))
suppressWarnings(suppressMessages(library(patchwork)))
suppressWarnings(suppressMessages(library(dplyr)))
suppressWarnings(suppressMessages(library(gridExtra)))
suppressWarnings(suppressMessages(library(RColorBrewer)))
suppressWarnings(suppressMessages(library(stringr)))
suppressWarnings(suppressMessages(library(scales)))
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(cowplot)))
suppressMessages(source(paste0(pipeline_dir, "/R/utils.R")))

In [3]:
#define colors for plotting
theme_set(theme_bw(base_size = 18) +
          theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()))
SpatialColors <- colorRampPalette(colors = rev(x = brewer.pal(n = 11, name = "Spectral")))

palette_length <- 100
my_palette <- SpatialColors(n = palette_length)
color_palette2 <- colorRampPalette(colors = rev(x = brewer.pal(n = 11, name = "Spectral")))

In [4]:
#reading manifest
sample_metadata <- read.csv(manifest_path)
head(sample_metadata, 3)

,alias_id,seq_sample_id,fw_sample_id,visium_slide,visium_capture_area,dual_index,eln,date_processed_lab,batch_group,diagnosis
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,1,1,9628,V12M09-090,A,1A,21316-916,4+5 july 2022,1,vehicle
2,2,2,9631,V12M09-090,B,1B,21316-916,4+5 july 2022,1,Sema
3,3,3,9675,V12M09-090,C,1C,21316-916,4+5 july 2022,1,Enalapril


In [8]:
# # #reading by sample
# sample_list <- sample_metadata$alias_id
# object_list <- vector('list', length(sample_list))
# for (i in 1:length(sample_list)) {
#     sample_name <- paste0("sample", sample_list[[i]])
#     curr_obj <- readRDS(file = file.path(processed_dir, "by_sample/rds/", paste0(sample_list[[i]], "_annotated_2.rds")))
    
#     if (length(curr_obj@assays$Spatial@scale.data) == 0){
#         # normalize and scale data so we can plot and color code
#         # curr_obj <- NormalizeData(curr_obj, assay = "Spatial")
#         # curr_obj <- ScaleData(curr_obj, assay = "Spatial", verbose = F)
#         message("Norm & scaling sample")
#     }

#     object_list[[i]] <- curr_obj
# }
# print("---")
# message("By sample objects read, first object looks like:")
# object_list[[1]]

#reading integrated
# integrated_obj <- readRDS(file.path(processed_dir, "integrated/rds/integrated_annotated_2.rds"))
integrated_obj$Region2 %>% unique()

[1] "Glomeruli"     "Cortex"        "ISOM"          "Inner_Medulla"
[5] "OSOM"          "Renal Pelvis"

In [16]:
# df <- FindSpatiallyVariableFeatures(object_list[[1]], assay="SCT", selection.method = "moransi", verbose = T, features = VariableFeatures(object_list[[1]])[1:1000])
#  SpatiallyVariableFeatures(object_list[[1]])
# head(SpatiallyVariableFeatures(df, selection.method = "moransi"),3)

Computing Moran's I



An object of class Seurat 
26128 features across 2747 samples within 2 assays 
Active assay: SCT (13064 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: Spatial
 2 dimensional reductions calculated: pca, umap
 1 image present: sample1

In [ ]:
#Comparing each region vs the rest to see if there are genes that are differentially expressed
Idents(integrated_obj) <- "Region2"
df_all <- FindAllMarkers(integrated_obj, assay = "Spatial", only.pos = F, min.pct=0, logfc.threshold = 0, test.use = "wilcox", verbose = T)
df <- split(df_all,df_all$cluster)

path <- ".//kidney_mouse_ntn/characterize_regions/"
#for each df in df, save the csv
for (i in 1:length(df)){
    write.csv(df[[i]], file = paste0(path,names(df)[i], ".csv"))

    #filtering 
    df[[i]] <- df[[i]] %>% 
        filter(p_val_adj < 0.05) %>%
        filter(pct.1 > 0.25)
    
    write.csv(df[[i]], file = paste0(path,names(df)[i], "_pval_pct1_filtered.csv"))
}

write.csv(df_all, file = paste0(path,"all_regions.csv"))

Calculating cluster Glomeruli

Calculating cluster Cortex

Calculating cluster ISOM

Calculating cluster Inner_Medulla

Calculating cluster OSOM

Calculating cluster Renal Pelvis



In [18]:
library(patchwork)

In [19]:
modify_vlnplot<- function(obj, 
                          feature, 
                          pt.size = 0, 
                          plot.margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                          ...) {
  p<- VlnPlot(obj, features = feature, pt.size = pt.size, ... )  + 
    xlab("") + ylab(feature) + ggtitle("") + 
    theme(legend.position = "none", 
          axis.text.x = element_blank(), 
          axis.ticks.x = element_blank(), 
          axis.title.y = element_text(size = rel(1), angle = 0), 
          axis.text.y = element_text(size = rel(1)), 
          plot.margin = plot.margin ) 
  return(p)
}

## extract the max value of the y axis
extract_max<- function(p){
  ymax<- max(ggplot_build(p)$layout$panel_scales_y[[1]]$range$range)
  return(ceiling(ymax))
}


## main function
StackedVlnPlot<- function(obj, features,
                          pt.size = 0, 
                          plot.margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                          ...) {
  
  plot_list<- purrr::map(features, function(x) modify_vlnplot(obj = obj,feature = x, ...))
  
  # Add back x-axis title to bottom plot. patchwork is going to support this?
  plot_list[[length(plot_list)]]<- plot_list[[length(plot_list)]] +
    theme(axis.text.x=element_text(), axis.ticks.x = element_line())
  
  # change the y-axis tick to only max value 
  ymaxs<- purrr::map_dbl(plot_list, extract_max)
  plot_list<- purrr::map2(plot_list, ymaxs, function(x,y) x + 
                            scale_y_continuous(breaks = c(y)) + 
                            expand_limits(y = y))

  p<- patchwork::wrap_plots(plotlist = plot_list, ncol = 1)
  return(p)
}


In [43]:
top_n <- df_all %>% 
    filter(pct.1 > 0.25) %>%
    filter(p_val_adj < 0.05) %>%
    #take top 10 per each cluster 
    group_by(cluster) %>%
    top_n(3, avg_log2FC) %>%
top_n

p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
0,3.832970,0.741,0.206,0,Glomeruli,Nphs1
0,3.796277,0.853,0.379,0,Glomeruli,Nphs2
0,3.881561,0.543,0.087,0,Glomeruli,Myom2
0,1.996380,0.698,0.381,0,Cortex,Slc5a12
0,1.952499,0.506,0.196,0,Cortex,Slc16a14
0,1.982949,0.700,0.444,0,Cortex,Pvalb
0,4.038267,0.780,0.299,0,ISOM,Nccrp1
0,3.835847,0.605,0.155,0,ISOM,Fst
0,4.269354,0.336,0.031,0,ISOM,Rph3a


In [ ]:
# features<- c("Ren1", "Ang", "Ace", "Ace2", "Agtr1a", "Mas1")
features <- c("Nphs1", "Nphs2", "Slc5a12", "Pvalb", "Nccrp1", "Fst", "Aldh1a3","Cldn11", "Igf2", "Napsa", "Cidec", "Cidea")
# p <- StackedVlnPlot(obj = integrated_obj, features = pull(top_n,gene))
p <- StackedVlnPlot(obj = integrated_obj, features = features)

ggsave(paste0(path, "vlnplot.png"), p, width = 10, height = 22, dpi = 300)